In [6]:
from operator import itemgetter
import json
import pandas as pd  
from nltk.tokenize import TweetTokenizer
from nltk import FreqDist

tknzr = TweetTokenizer()


# read all sample tweets.
with open('all_stripped_tweets.json', "r", encoding="utf-8") as f:  
    records=[json.loads(line) for line in f]
    

# combine relevant details into one list
data = []

for i in range(0,len(records)):
    topic = records[i]['topic']
    text = records[i]['text']
    comb = [text,topic]   
    data.append(comb)
    
# pass list to dataframe
df = pd.DataFrame(data,columns=['text','topic'])

    
def get_hashtags(topic):
    big_string = []
    df_hash = df[df['topic']==topic]
    for i in range(1,len(df_hash)):
        big_string.append(df_hash.iloc[i][0])

    big_string  = str(big_string)
    toked = tknzr.tokenize(big_string)
    hashtags = [hashtags for hashtags in toked if '#' in hashtags]
    hash_dist = FreqDist(w.lower() for w in hashtags) 
    hash_dist_sorted = sorted(hash_dist.items(), key=itemgetter(1),reverse=True)
    
    # set a threshold for new hashtags - take top 10% by tweet count or those with >5 tweets
    if (hash_dist_sorted[0][1]/10)>=5:
        count_threshold = hash_dist_sorted[0][1]/10
    else:
        count_threshold = 5

    #print("Topic: ",topic," - Threshold set at: ",count_threshold)#,"\n\n")

    for j in range(0,10):
        if hash_dist_sorted[j][1] >= count_threshold:
            set_count = j

    df_topic_tags = pd.DataFrame(hash_dist_sorted[0:set_count+1], columns=['Hashtag','Freq'])
    df_topic_tags['topic'] = topic
        
    return df_topic_tags


########################################################################


# count tweets in example, sort and slice top 15
df_topics = df.groupby(by="topic").count()
df_topics = df_topics.sort_values('text',ascending=False)
df_top_topics = df_topics.iloc[0:15]

all_topic_tags = pd.DataFrame()


# step through the 15 topics and retreive the related hashtags - updating the dataframe
for row in df_top_topics.T.iteritems():
    topic = row[0]
    topic_tags = get_hashtags(topic)
    all_topic_tags = pd.concat([all_topic_tags,topic_tags])


# bulking out static topics with previous list
with open('ads_topics.json') as data_file:    
    data = json.load(data_file)


for name in range(0,len(data)):
    top = data[name]['name']
    topic_list_1 = data[name]["queries"]
    topic_list_2 = all_topic_tags[all_topic_tags['topic']==top]['Hashtag'].tolist()
    data[name].update({'queries':topic_list_1+topic_list_2})
    

with open('ads_topics_NEW.json', 'w') as outfile:
    json.dump(data, outfile)

    
data

[{'name': 'Trump',
  'queries': ['trump',
   '@realDonaldTrump',
   '@POTUS',
   'muslimban',
   'fakenews',
   '#americafirst',
   '#obamagate',
   '#trumptrain',
   '#lockherup',
   '#trump',
   '#fakenews',
   '#trumpet',
   '#brexit',
   '#instagram',
   '#inspire',
   '#love',
   '#london',
   '#terrorist',
   '#londonattacks']},
 {'name': 'Russia',
  'queries': ['#russia',
   '#putin',
   '#syria',
   '#moscow',
   '#kabul',
   '#nato',
   '#afganistan',
   '#isis',
   '#ukraine',
   '#madvlad']},
 {'name': 'Technology',
  'queries': ['#ArtificialIntelligence',
   '#machinelearning',
   '#deeplearning',
   '@DeepMindAI',
   '#Xboxone',
   '#Xbox360',
   '@Xbox',
   '#xbox',
   '#ps4',
   '#ps4pro',
   '@Playstation',
   '@PlayStationUK',
   '#google',
   '#elonmusk',
   '#spacex',
   '#robots',
   '#ces',
   '#gadgets',
   '#robotics',
   '#robots',
   '#bigdata',
   '#3dprinting',
   '#data',
   '#datascience',
   '#tech',
   '#tech',
   '#data',
   '#technology',
   '#ai',
   '